In [1]:
import findspark
import os
from pyspark.sql import SparkSession, DataFrame

os.environ["SPARK_HOME"] = "/home/nphung/spark-3.3.1-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"
os.environ["PYSPARK_PYTHON"] = "python3.10"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3.10"

findspark.init()

spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.driver.memory", "2G")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .getOrCreate()
)

print("Spark session is now available")

your 131072x1 screen size is bogus. expect trouble


23/01/26 20:06:51 WARN Utils: Your hostname, DESKTOP-9EFE6HH resolves to a loopback address: 127.0.1.1; using 172.24.61.140 instead (on interface eth0)
23/01/26 20:06:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/26 20:06:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/26 20:06:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark session is now available


In [2]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import (
    ArrayType,
    LongType,
    StringType,
    StructField,
    StructType,
    StringType,
    DoubleType,
    IntegerType,
)

transaction: DataFrame = spark.createDataFrame(
    [
        ("01/01/20", 1234, 999, 490756, 50.0, 1),
        ("01/01/20", 1234, 999, 389728, 3.56, 4),
        ("01/01/20", 3456, 845, 490756, 50.0, 2),
        ("01/01/20", 3456, 845, 549380, 300.0, 1),
        ("01/01/20", 3456, 845, 293718, 10.0, 6),
    ],
    StructType(
        [
            StructField("date", StringType(), False),
            StructField("order_id", LongType(), False),
            StructField("client_id", LongType(), False),
            StructField("prod_id", LongType(), False),
            StructField("prod_price", DoubleType(), False),
            StructField("prod_qty", IntegerType(), False),
        ]
    ),
)

transaction.createOrReplaceTempView("transaction")

In [3]:
product_nomenclature: DataFrame = spark.createDataFrame(
    [
        (
            490756,
            "MEUBLE",
            "Chaise",
        ),
        (
            389728,
            "DECO",
            "Boule de Noël",
        ),
        (
            549380,
            "MEUBLE",
            "Canapé",
        ),
        (
            293718,
            "MEUBLE",
            "Mug",
        ),
    ],
    StructType(
        [
            StructField("product_id", LongType(), False),
            StructField("product_type", StringType(), False),
            StructField("product_name", StringType(), False),
        ]
    ),
)

product_nomenclature.createOrReplaceTempView("product_nomenclature")

In [4]:
spark.sql(
    """SELECT * FROM product_nomenclature 
    INNER JOIN transaction where transaction.prod_id=product_nomenclature.product_id
"""
)

product_id,product_type,product_name,date,order_id,client_id,prod_id,prod_price,prod_qty
490756,MEUBLE,Chaise,01/01/20,1234,999,490756,50.0,1
389728,DECO,Boule de Noël,01/01/20,1234,999,389728,3.56,4
490756,MEUBLE,Chaise,01/01/20,3456,845,490756,50.0,2
549380,MEUBLE,Canapé,01/01/20,3456,845,549380,300.0,1
293718,MEUBLE,Mug,01/01/20,3456,845,293718,10.0,6


In [5]:
# Get the total prices of product per day from 01/01/2020 to 31/12/2020, sort by day

spark.sql(
    """SELECT date, sum(prod_price*prod_qty) as total FROM product_nomenclature 
    INNER JOIN transaction ON transaction.prod_id=product_nomenclature.product_id
    WHERE date='01/01/20'
    GROUP BY date
    ORDER BY date
"""
)

date,total
01/01/20,524.24


In [6]:
# Get product_type total by client_id 

spark.sql(
    """SELECT client_id,product_type, SUM(prod_price*prod_qty) as ventes  FROM product_nomenclature 
    INNER JOIN transaction where transaction.prod_id=product_nomenclature.product_id
    GROUP BY client_id, product_type
"""
)

client_id,product_type,ventes
999,MEUBLE,50.0
845,MEUBLE,460.0
999,DECO,14.24


In [7]:
# Total per customer regroup by product_type for a give transaction period between two dates

spark.sql(
    """
    SELECT client_id, 
            sum(case when product_type="MEUBLE" then ventes end) as ventes_meuble,
            sum(case when product_type="DECO" then ventes end) as ventes_deco
      FROM (
        SELECT client_id,product_type, SUM(prod_price*prod_qty) as ventes  FROM product_nomenclature 
        INNER JOIN transaction ON transaction.prod_id=product_nomenclature.product_id
        WHERE date='01/01/20'
        GROUP BY client_id, product_type
    ) GROUP BY client_id
"""
)

client_id,ventes_meuble,ventes_deco
999,50.0,14.24
845,460.0,null
